In [2]:
# Project Libraries
import data

# Generic Libraries
import numpy as np
import pandas as pd

Data Frames:
- Orderbook with previous operations
- Liquidity pools history (deliverable)
- Trade History (deliverable) 

In [3]:
# ----- Useful dataframes

def dataframes():
    """
    L

    Parameters
    ----------

    L

    Returns
    -------

    L
    
    """
    
    # Informative dataframe
    df_info = pd.DataFrame({
        'bid_volume': [0.0001],
        'ask_volume': [0.0001],
        'commission': [0.000025]
    })

    # Oders dataframe
    df_orders = pd.DataFrame(columns = ['timestamp', 'order_id', 'side', 'price', 'order_amount', 'symbol'])

    # Trades dataframe
    df_trades = pd.DataFrame(columns = ['timestamp', 'order_id', 'trade_id', 'side', 'price', 'symbol', 'filled_amount', 'order_amount', 'fee'])

    # Inventory dataframe
    df_inventory = pd.DataFrame({
        'timestamp': [0],
        'base (BTC)': [5],
        'quote (USDT)': [500000]
    })

    # Rebalance dataframe
    df_rebalances = pd.DataFrame(columns = ['timestamp', 'sent', 'received', 'price', 'fee'])

    return df_info, df_orders, df_trades, df_inventory, df_rebalances


In [4]:
def order_id(col):
    if col == 'order_id':
        try:
            return df_orders['order_id'].iloc[-1] + 1
        except:
            return 1
    if col == 'index':
        try:
            return df_orders.index[-1] + 1
        except:
            return 0

In [5]:
# ----- Order Placement Function
def order_placement(df, ts, position, order_id_passed):
    i = order_id('index')
    if position == 'buy':
        df_orders.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
        df_orders.at[i, 'order_id'] = order_id_passed
        df_orders.at[i, 'side'] = 'buy' 
        df_orders.at[i, 'price'] = df['bid'].iloc[ts]
        df_orders.at[i, 'order_amount'] = 0.0001
        df_orders.at[i, 'symbol'] = 'BTC/USDT'
    else:
        df_orders.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
        df_orders.at[i, 'order_id'] = order_id_passed
        df_orders.at[i, 'side'] = 'sell'
        df_orders.at[i, 'price'] = df['bid'].iloc[ts]       
        df_orders.at[i, 'order_amount'] = 0.0001
        df_orders.at[i, 'symbol'] = 'BTC/USDT'

In [6]:
# Position valuation
def filled_volume(size_variation: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation > 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    # Bid
    if operation == 'bid':
        if price_variation < 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    return current_position

In [7]:
def trade_id(col):
    if col == 'trade_id':
        try:
            return df_trades['trade_id'].iloc[-1] + 1
        except:
            return 1
    if col == 'index':
        try:
            return df_trades.index[-1] + 1
        except:
            return 0

In [80]:
def trade(df, ts, position, trade_id_passed, filled_volume):

    i = trade_id('index')
    traded_price = df_orders[df_orders['side'] == position].iloc[-1]['price']
    ordered_amount = df_orders[df_orders['side'] == position].iloc[-1]['order_amount']
    commision = df_info['commission'].iloc[0]

    df_trades.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
    df_trades.at[i, 'order_id'] = df_orders[df_orders['side'] == position].iloc[-1]['order_id']
    df_trades.at[i, 'trade_id'] = trade_id_passed
    df_trades.at[i, 'side'] = position
    df_trades.at[i, 'price'] = traded_price
    df_trades.at[i, 'symbol'] = 'BTC/USDT'
    df_trades.at[i, 'filled_amount'] = filled_volume
    df_trades.at[i, 'order_amount'] = ordered_amount
    df_trades.at[i, 'fee'] = traded_price * ordered_amount * commision


In [9]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})


In [38]:
pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index().iloc[0]['ask']

{'0': 28275.0,
 '1': 28276.0,
 '2': 28277.0,
 '3': 28278.0,
 '4': 28280.0,
 '5': 28281.0,
 '6': 28282.0,
 '7': 28283.0,
 '8': 28284.0,
 '9': 28285.0,
 '10': 28286.0,
 '11': 28287.0,
 '12': 28288.0,
 '13': 28289.0,
 '14': 28290.0,
 '15': 28291.0,
 '16': 28292.0,
 '17': 28293.0,
 '18': 28294.0,
 '19': 28295.0,
 '20': 28296.0,
 '21': 28297.0,
 '22': 28298.0,
 '23': 28299.0,
 '24': 28300.0}

In [47]:
df_bitfinex_tob['bid'].apply(lambda x: x-28282).unique()

array([  0.,  41.,  27.,  47.,  54.,  60., 106., 100.,  88.,  80.,  55.,
        30.,  16.,   7.,  20.,   9.,  17.,  14.,  58.,  78.,  64.,  56.,
        53.,  76.,  75.,  74.,  62.,  72.,  71., 105., 101.,  98., 137.,
       156., 153., 134., 131., 127., 119., 121., 125.,  69.,  51.,   8.,
        -8.,   6.,  13.,  73.])

In [85]:
#df_info, df_orders, df_trades, df_inventory, df_rebalances = fn.dataframes()
df_info, df_orders, df_trades, df_inventory, df_rebalances = dataframes()

order_placement(df = df_bitfinex_tob,
                ts = 0,  
                position = "buy",
                order_id_passed = order_id("order_id"))
order_placement(df = df_bitfinex_tob, 
                ts = 0,
                position = "sell",
                order_id_passed = order_id("order_id"))

filled = []

for iteration in range(1,len(df_bitfinex_tob)):
    for op in ['ask']:
        buy_sell = ('buy' if op == 'bid' else 'sell')
        if len(df_trades[df_trades['side']==buy_sell]) == 0:
            filled_amount = filled_volume(
                size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
                price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
                operation = op
                )
        else:
            filled_amount = filled_volume(
                size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
                price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
                operation = op,
                constant_position_base = df_orders[df_orders['side'] == buy_sell].iloc[-1]['order_amount'] - df_trades[df_trades['side']==buy_sell]['filled_amount'].iloc[-1]
                )
        filled.append(filled_amount)
        if filled_amount > 0:
                print('Trade:',op)
                trade( df = df_bitfinex_tob,
                    ts = trade_id('index'),
                    position = buy_sell, 
                    trade_id_passed = trade_id('trade_id'), 
                    filled_volume = filled_amount
                    )             
                position_trades = df_trades.copy()
                position_trades = position_trades[position_trades['side'] == buy_sell]
                if len(position_trades) > 0:
                        if position_trades['filled_amount'].iloc[-1] == position_trades['order_amount'].iloc[-1]:
                            print(position_trades['filled_amount'].iloc[-1])
                            order_placement(df = df_bitfinex_tob, 
                                            ts = iteration, 
                                            position = op,
                                            order_id_passed = order_id("order_id"))
df_trades


Trade: ask


,timestamp,order_id,trade_id,side,price,symbol,filled_amount,order_amount,fee
0,2021-07-05 13:06:00+00:00,2,1,sell,28282.0,BTC/USDT,0.0001,0.0001,0.000071


In [88]:
df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,1,buy,28282.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,2,sell,28282.0,0.0001,BTC/USDT
2,2021-07-05 13:07:00+00:00,3,sell,28323.0,0.0001,BTC/USDT
3,2021-07-05 13:08:00+00:00,4,sell,28309.0,0.0001,BTC/USDT
4,2021-07-05 13:09:00+00:00,5,sell,28329.0,0.0001,BTC/USDT
...,...,...,...,...,...,...
57,2021-07-05 14:02:00+00:00,58,sell,28288.0,0.0001,BTC/USDT
58,2021-07-05 14:03:00+00:00,59,sell,28295.0,0.0001,BTC/USDT
59,2021-07-05 14:04:00+00:00,60,sell,28340.0,0.0001,BTC/USDT
60,2021-07-05 14:05:00+00:00,61,sell,28355.0,0.0001,BTC/USDT


In [33]:
#df_info, df_orders, df_trades, df_inventory, df_rebalances = fn.dataframes()
df_info, df_orders, df_trades, df_inventory, df_rebalances = dataframes()

order_placement(df = df_bitfinex_tob,
                ts = 0,  
                position = "buy",
                order_id_passed = order_id("order_id"))
order_placement(df = df_bitfinex_tob, 
                ts = 0,
                position = "sell",
                order_id_passed = order_id("order_id"))

for iteration in range(1,len(df_bitfinex_tob)):

    for op in ['ask','bid']:
        buy_sell = ('buy' if op == 'bid' else 'sell')
        if len(df_trades[df_trades['side']==buy_sell]) == 0:
            filled_amount = filled_volume(
                size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
                price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
                operation = op
                )
        else:
            filled_amount = filled_volume(
                size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
                price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
                operation = op,
                constant_position_base = df_orders[df_orders['side'] == buy_sell].iloc[-1]['order_amount'] - df_trades[df_trades['side']==buy_sell]['filled_amount'].iloc[iteration - 1]
                )
        
        if filled_amount > 0:
            print('Trade:',op)
            trade( df = df_bitfinex_tob,
                ts = iteration,
                position = buy_sell, 
                trade_id_passed = trade_id('trade_id'), 
                filled_volume = filled_amount
                )

        position_trades = df_trades.copy()
        position_trades = position_trades[position_trades['side'] == buy_sell]

        if len(position_trades) > 0:
            if position_trades['filled_amount'].iloc[-1] == position_trades['order_amount'].iloc[-1]:
                order_placement(df = df_bitfinex_tob, 
                                ts = iteration, 
                                position = op,
                                order_id_passed = order_id("order_id"))



Trade: ask


IndexError: single positional indexer is out-of-bounds

In [283]:
filled_amount = position_valuation(
        size_variation  = df_bitfinex_tob[op+'_size'].iloc[1] - df_bitfinex_tob[op+'_size'].iloc[1],
        price_variation = df_bitfinex_tob[op].iloc[1] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
        operation = op
        )
filled_amount

0.0001

In [287]:
pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index().iloc[0]['ask']

{'0': 28275.0,
 '1': 28276.0,
 '2': 28277.0,
 '3': 28278.0,
 '4': 28280.0,
 '5': 28281.0,
 '6': 28282.0,
 '7': 28283.0,
 '8': 28284.0,
 '9': 28285.0,
 '10': 28286.0,
 '11': 28287.0,
 '12': 28288.0,
 '13': 28289.0,
 '14': 28290.0,
 '15': 28291.0,
 '16': 28292.0,
 '17': 28293.0,
 '18': 28294.0,
 '19': 28295.0,
 '20': 28296.0,
 '21': 28297.0,
 '22': 28298.0,
 '23': 28299.0,
 '24': 28300.0}

In [264]:
df_bitfinex_tob

,timestamp,ask_size,ask,bid,bid_size
0,2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172
1,2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797
2,2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481
3,2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650
4,2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731
...,...,...,...,...,...
56,2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627
57,2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127
58,2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682
59,2021-07-05 14:05:00+00:00,0.445649,28361.0,28355.0,0.006000


In [272]:
iteration = 1
op = 'bid'
buy_sell = ('buy' if op == 'bid' else 'sell')
filled_amount = position_valuation(
    size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
    price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
    operation = op,
    constant_position_base = df_orders[df_orders['side'] == buy_sell].iloc[0]['order_amount'] - df_trades['filled_amount'].iloc[iteration - 1]
    )
filled_amount

0.0001

In [ ]:
iteration = 2

for op in ['bid','ask']:
    buy_sell = ('buy' if op == 'bid' else 'sell')
    filled_amount = position_valuation(
        size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
        price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[-1]['price'],
        operation = op,
        constant_position_base = df_orders[df_orders['side'] == buy_sell].iloc[0]['order_amount'] - df_trades['filled_amount'].iloc[iteration - 1]
        )

    trade( df = df_bitfinex_tob,
        ts = iteration,
        position = buy_sell, 
        trade_id_passed = trade_id('trade_id'), 
        filled_volume = filled_amount
        )
    position_trades = df_trades.copy()
    position_trades = position_trades[position_trades['side'] == buy_sell]
    if len(position_trades) > 0:
        if position_trades['filled_amount'].iloc[-1] == position_trades['order_amount'].iloc[-1]:
            order_placement(df = df_bitfinex_tob, 
                            ts = iteration, 
                            position = op,
                            order_id_passed = order_id("order_id"))

In [225]:
df_bitfinex_tob

,timestamp,ask_size,ask,bid,bid_size
0,2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172
1,2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797
2,2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481
3,2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650
4,2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731
...,...,...,...,...,...
56,2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627
57,2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127
58,2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682
59,2021-07-05 14:05:00+00:00,0.445649,28361.0,28355.0,0.006000


In [61]:
# Explanation Missing
def position_valuation(size: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation >= 0:
            if ( constant_position_base - size ) < 0:
                current_position = 0
            else:
                current_position = constant_position_base - size
        else:
            current_position = constant_position_base
    # Bid
    if operation == 'bid':
        if price_variation <= 0:
            if ( constant_position_base - size ) < 0:
                current_position = 0
            else:
                current_position = constant_position_base - size
        else:
            current_position = constant_position_base
    return current_position